# Analisis de prestaciones por diagnostico

- En este apartado sequiere ver la cantidad de prestaciones que se realizan por cada diagnostico principal.

- El resultado que se quiere obtener es parecido al siguiente:

| Año  | Diagnostico | Resumen Procedimientos     |
| ---- | ----------- | -------------------------- |
| 2019 | "I25.1"     | {"45.23": 20, "73.59": 10} |
| 2020 | "I25.1"     | {"45.23": 7, "73.59": 3}   |


In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103

## 1. Análisis de Procedimientos a Nivel País

En este análisis se quieren responder las siguiente preguntas:

1. ¿Cómo es el ranking de los hospitales para cada uno de los procedimientos realizads?
2. ¿Quién es el Hospital que realiza más de X procedimientos?
3. ¿El procedimiento X es realizado en todo el país? ¿En qué partes en específico se realiza?

Para responder esta pregunta se utilizará la base de datos de Egresos de Hospitales adscritos
a GRD entre 2019 y 2021.

In [3]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";")

In [5]:
display(df_procesada.head(10))

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA
0,118100,1314867.0,MUJER,1988-06-19,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,1.0,146102.0,0.5744,2.0,1.0,NaN,5.0,2019.0,3.0,2019-3
1,118100,1354418.0,MUJER,1988-03-27,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,NaN,134161.0,0.2951,1.0,1.0,NaN,4.0,2019.0,3.0,2019-3
2,118100,239861.0,MUJER,2017-06-17,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,64182.0,0.6736,2.0,2.0,NaN,6.0,2019.0,3.0,2019-3
3,118100,330326.0,HOMBRE,2017-12-04,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,NaN,64172.0,0.5475,2.0,2.0,NaN,2.0,2019.0,3.0,2019-3
4,118100,1369293.0,MUJER,1995-12-06,OTRO,CONCEPCION,FLORIDA,CHILE,FONASA INSTITUCIONAL - (MAI) C,CONCEPCIÓN,...,NaN,134141.0,0.3107,1.0,1.0,NaN,6.0,2019.0,3.0,2019-3
5,118100,469385.0,MUJER,1942-06-29,OTRO,CONCEPCION,CHIGUAYANTE,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,114103.0,1.1799,3.0,3.0,NaN,47.0,2019.0,3.0,2019-3
6,118100,1146635.0,MUJER,1951-07-14,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,44163.0,1.0328,3.0,3.0,NaN,43.0,2019.0,3.0,2019-3
7,118100,93557.0,HOMBRE,1938-03-04,OTRO,CONCEPCION,LOTA,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,1.0,11101.0,1.8731,1.0,1.0,NaN,41.0,2019.0,3.0,2019-3
8,118100,674943.0,MUJER,1962-08-21,OTRO,CONCEPCION,SANTA JUANA,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,1.0,61402.0,1.3078,2.0,2.0,HOSPITAL CLORINDA AVELLO (SANTA JUANA),33.0,2019.0,3.0,2019-3
9,118100,1079955.0,HOMBRE,1944-12-07,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,14133.0,1.6687,3.0,3.0,NaN,31.0,2019.0,3.0,2019-3


Los procedimientos en esta base de datos se encuentran en las columnas "PROCEDIMIENTOSX", donde X
va desde 1 hasta 30. Para analizar esta base de forma más expedita, se transformará de formato
ancho a formato largo, utilizando como identificador a las columnas ["COD_HOSPITAL", "ANIO_EGRESO", 
"CIP_ENCRIPTADO", "DIAGNOSTICO1"]. Cabe destacar que la columna "CIP_ENCRIPTADO" hace referencia al
RUT del paciente.

In [8]:
procedimientos_long = pd.melt(
    df_procesada,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "CIP_ENCRIPTADO", "DIAGNOSTICO1"],
    value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)],
)

In [18]:
conteo_procedimientos = (
    procedimientos_long.groupby(["ANIO_EGRESO", "COD_HOSPITAL"])["value"]
    .value_counts()
    .reset_index(name="conteo")
    .sort_values(["ANIO_EGRESO", "value", "conteo"], ascending=False)
)

conteo_procedimientos["ranking"] = (
    conteo_procedimientos.groupby(["ANIO_EGRESO", "value"]).cumcount() + 1
)

conteo_procedimientos["diferencia_proximo_hospital"] = conteo_procedimientos["conteo"].diff(
    periods=-1
)

conteo_procedimientos["diferencia_proximo_hospital"] = np.where(
    conteo_procedimientos["diferencia_proximo_hospital"] < 0,
    -1,
    conteo_procedimientos["diferencia_proximo_hospital"],
)

In [19]:
mejores_procedimientos_torax = conteo_procedimientos.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX and ranking == 1"
)

In [54]:
conteo_procedimientos.query("value == 35.05")

,ANIO_EGRESO,COD_HOSPITAL,value,conteo,ranking,diferencia_proximo_hospital
178208,2021.0,118100,35.05,19,1,1.0
153623,2021.0,110100,35.05,18,2,5.0
162336,2021.0,112103,35.05,13,3,4.0
179991,2021.0,119100,35.05,9,4,3.0
147813,2021.0,107100,35.05,6,5,0.0
166735,2021.0,114101,35.05,6,6,1.0
139732,2021.0,103100,35.05,5,7,0.0
158914,2021.0,111195,35.05,5,8,1.0
151659,2021.0,109100,35.05,4,9,2.0
190325,2021.0,126100,35.05,2,10,1.0


In [7]:
mejores_procedimientos_torax.query("diferencia_proximo_hospital == -1")

,ANIO_EGRESO,COD_HOSPITAL,value,conteo,ranking,diferencia_proximo_hospital
162597,2021.0,112103,87.93,1,1,-1.0
162525,2021.0,112103,32.21,1,1,-1.0
99861,2020.0,112103,40.69,1,1,-1.0
99661,2020.0,112103,37.6,5,1,-1.0
99837,2020.0,112103,37.37,1,1,-1.0
99732,2020.0,112103,33.48,2,1,-1.0
